In [ ]:
# API Usage Guide - Real-time Fraud Detection

This notebook demonstrates how to use the Enterprise Fraud Detection API for real-time fraud scoring and integration patterns.

## Table of Contents

1. [API Overview](#overview)
2. [Authentication & Setup](#auth)
3. [Single Transaction Scoring](#single)
4. [Batch Processing](#batch)
5. [Integration Patterns](#integration)
6. [Performance Testing](#performance)
7. [Error Handling](#errors)

---

## Introduction

The Fraud Detection API provides real-time fraud scoring capabilities through RESTful endpoints. It processes transactions in real-time and returns comprehensive risk assessments with recommended actions.

### API Endpoints:
- `POST /fraud-detection` - Single transaction scoring
- `POST /fraud-detection/batch` - Batch transaction processing  
- `GET /health` - System health check
- `GET /metrics` - Performance metrics
- `GET /models/info` - Model information


In [ ]:
# Setup and imports
import sys
import os
import pandas as pd
import numpy as np
import requests
import json
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

# Add the src directory to Python path
sys.path.append('../src')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🔧 API Usage Environment Setup Complete")
print(f"📅 Session Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# API Configuration
API_BASE_URL = "http://localhost:8000"  # Fraud Detection API
API_TIMEOUT = 30

print(f"🌐 API Base URL: {API_BASE_URL}")
print(f"⏱️ Request Timeout: {API_TIMEOUT}s")


In [ ]:
## API Health Check

Before using the fraud detection endpoints, let's check if the API is healthy and running correctly.


In [ ]:
def check_api_health():
    """Check API health and return status"""
    try:
        response = requests.get(f"{API_BASE_URL}/health", timeout=API_TIMEOUT)
        
        if response.status_code == 200:
            health_data = response.json()
            print("✅ API Health Check - HEALTHY")
            print("=" * 35)
            print(f"Status: {health_data['status']}")
            print(f"Timestamp: {health_data['timestamp']}")
            
            print("\n📊 Component Status:")
            for component, status in health_data['components'].items():
                emoji = "✅" if status == "healthy" or status == "loaded" else "❌"
                print(f"  {emoji} {component}: {status}")
            
            if 'performance_metrics' in health_data:
                metrics = health_data['performance_metrics']
                print(f"\n🚀 Performance Metrics:")
                print(f"  Total Requests: {metrics['total_requests']:,}")
                print(f"  Avg Response Time: {metrics['average_processing_time_ms']:.1f}ms")
                print(f"  Requests/Second: {metrics['requests_per_second']:.1f}")
            
            return True
        else:
            print(f"❌ API Health Check Failed - Status: {response.status_code}")
            return False
            
    except requests.exceptions.ConnectionError:
        print("❌ API Health Check Failed - Connection Error")
        print("💡 Make sure the API is running: python -m src.inference.api")
        return False
    except Exception as e:
        print(f"❌ API Health Check Failed - Error: {e}")
        return False

# Simulate API health check (since API might not be running)
def simulate_health_check():
    """Simulate a successful health check response"""
    print("🔄 Simulating API Health Check...")
    
    simulated_response = {
        "status": "healthy",
        "timestamp": datetime.now().isoformat(),
        "components": {
            "database": "healthy",
            "redis": "healthy", 
            "hub_model": "loaded",
            "spoke_models": {
                "pix": "loaded",
                "credit_card": "loaded",
                "loan": "loaded"
            }
        },
        "performance_metrics": {
            "total_requests": 15847,
            "average_processing_time_ms": 85.3,
            "requests_per_second": 127.5,
            "timestamp": datetime.now().isoformat()
        }
    }
    
    print("✅ API Health Check - HEALTHY (Simulated)")
    print("=" * 45)
    print(f"Status: {simulated_response['status']}")
    print(f"Timestamp: {simulated_response['timestamp']}")
    
    print("\n📊 Component Status:")
    for component, status in simulated_response['components'].items():
        if isinstance(status, dict):
            print(f"  📁 {component}:")
            for sub_comp, sub_status in status.items():
                emoji = "✅" if sub_status == "loaded" else "❌"
                print(f"    {emoji} {sub_comp}: {sub_status}")
        else:
            emoji = "✅" if status == "healthy" else "❌"
            print(f"  {emoji} {component}: {status}")
    
    metrics = simulated_response['performance_metrics']
    print(f"\n🚀 Performance Metrics:")
    print(f"  Total Requests: {metrics['total_requests']:,}")
    print(f"  Avg Response Time: {metrics['average_processing_time_ms']:.1f}ms")
    print(f"  Requests/Second: {metrics['requests_per_second']:.1f}")

# Try real health check first, fallback to simulation
api_available = check_api_health()
if not api_available:
    print("\n" + "="*50)
    simulate_health_check()


In [ ]:
## Single Transaction Fraud Detection

The main fraud detection endpoint processes individual transactions and returns comprehensive risk assessments. Let's demonstrate how to use it with different types of transactions.


In [ ]:
def detect_fraud_single(transaction_data, include_explanation=False):
    """Send single transaction for fraud detection"""
    
    endpoint = f"{API_BASE_URL}/fraud-detection"
    params = {"include_explanation": include_explanation} if include_explanation else {}
    
    try:
        response = requests.post(
            endpoint, 
            json=transaction_data,
            params=params,
            timeout=API_TIMEOUT
        )
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ API Error - Status: {response.status_code}")
            print(f"Response: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Request Error: {e}")
        return None

def simulate_fraud_detection(transaction_data, include_explanation=False):
    """Simulate fraud detection response"""
    
    # Simulate processing time
    processing_time = np.random.uniform(50, 150)
    
    # Generate realistic scores based on transaction characteristics
    base_hub_score = 0.3
    base_spoke_score = 0.25
    
    # Adjust scores based on transaction features
    if transaction_data.get('amount', 0) > 5000:
        base_hub_score += 0.2
        base_spoke_score += 0.3
    
    if transaction_data.get('channel') == 'atm':
        base_spoke_score += 0.1
    
    # Weekend or night transactions are riskier
    hour = datetime.now().hour
    if hour < 6 or hour > 22:
        base_spoke_score += 0.15
    
    hub_score = min(1.0, base_hub_score + np.random.normal(0, 0.1))
    spoke_score = min(1.0, base_spoke_score + np.random.normal(0, 0.1))
    final_score = hub_score * 0.4 + spoke_score * 0.6
    
    # Determine risk level and action
    if final_score > 0.8:
        risk_level = "high"
        action = "reject"
        predicted_class = "fraud"
    elif final_score > 0.5:
        risk_level = "medium"
        action = "challenge"
        predicted_class = "legitimate"
    else:
        risk_level = "low"
        action = "approve"
        predicted_class = "legitimate"
    
    # Generate reason codes
    reason_codes = []
    if transaction_data.get('amount', 0) > 5000:
        reason_codes.append("HIGH_AMOUNT")
    if hour < 6 or hour > 22:
        reason_codes.append("UNUSUAL_TIME_OF_DAY")
    if transaction_data.get('channel') == 'atm':
        reason_codes.append("ATM_TRANSACTION")
    
    response = {
        "transaction_id": transaction_data["transaction_id"],
        "customer_id": transaction_data["customer_id"],
        "product_type": transaction_data["product_type"],
        "hub_risk_score": round(hub_score, 3),
        "spoke_fraud_score": round(spoke_score, 3),
        "final_score": round(final_score, 3),
        "risk_level": risk_level,
        "predicted_class": predicted_class,
        "confidence": round(max(final_score, 1 - final_score), 3),
        "action": action,
        "reason_codes": reason_codes,
        "processing_time_ms": round(processing_time, 1),
        "model_versions": {
            "hub_model": "v2.1.0",
            "spoke_model": f"{transaction_data['product_type']}_v1.3.0"
        },
        "timestamp": datetime.now().isoformat()
    }
    
    if include_explanation:
        response["explanation"] = {
            "summary": f"Customer risk level: {hub_score:.2f}, Transaction risk: {spoke_score:.2f}",
            "hub_model_contribution": f"{hub_score * 0.4:.3f} (40% weight)",
            "spoke_model_contribution": f"{spoke_score * 0.6:.3f} (60% weight)",
            "final_score_calculation": f"Final score: {final_score:.3f}",
            "key_risk_factors": reason_codes[:3]
        }
    
    return response

# Create sample transactions for different scenarios
sample_transactions = [
    {
        "transaction_id": "txn_normal_001",
        "customer_id": "cust_123456",
        "product_type": "pix",
        "amount": 150.00,
        "currency": "BRL",
        "channel": "mobile_app",
        "timestamp": datetime.now().isoformat(),
        "device_id": "device_mobile_123",
        "ip_address": "192.168.1.100",
        "beneficiary_id": "benef_789"
    },
    {
        "transaction_id": "txn_suspicious_002", 
        "customer_id": "cust_789012",
        "product_type": "credit_card",
        "amount": 8500.00,
        "currency": "BRL",
        "channel": "web_browser",
        "timestamp": datetime.now().isoformat(),
        "device_id": "device_web_456",
        "ip_address": "10.0.0.50",
        "merchant_id": "merchant_electronics",
        "merchant_category": "5732"
    },
    {
        "transaction_id": "txn_risky_003",
        "customer_id": "cust_345678", 
        "product_type": "ted",
        "amount": 25000.00,
        "currency": "BRL",
        "channel": "atm",
        "timestamp": datetime.now().isoformat(),
        "beneficiary_id": "benef_unknown_999"
    }
]

print("🔍 Single Transaction Fraud Detection Examples")
print("=" * 50)

for i, transaction in enumerate(sample_transactions, 1):
    print(f"\n📋 Example {i}: {transaction['transaction_id']}")
    print(f"   Product: {transaction['product_type'].upper()}")
    print(f"   Amount: R$ {transaction['amount']:,.2f}")
    print(f"   Channel: {transaction['channel']}")
    
    # Try real API first, fallback to simulation
    if api_available:
        result = detect_fraud_single(transaction, include_explanation=True)
    else:
        result = simulate_fraud_detection(transaction, include_explanation=True)
    
    if result:
        print(f"\n   📊 Results:")
        print(f"   Final Score: {result['final_score']:.3f}")
        print(f"   Risk Level: {result['risk_level'].upper()}")
        print(f"   Action: {result['action'].upper()}")
        print(f"   Processing Time: {result['processing_time_ms']}ms")
        if result['reason_codes']:
            print(f"   Reason Codes: {', '.join(result['reason_codes'])}")
    
    print("-" * 50)


In [ ]:
## Batch Transaction Processing

For high-volume scenarios, the API provides batch processing capabilities that can handle up to 100 transactions in a single request, processing them in parallel for optimal performance.


In [ ]:
def detect_fraud_batch(transactions, include_explanation=False):
    """Send batch of transactions for fraud detection"""
    
    endpoint = f"{API_BASE_URL}/fraud-detection/batch"
    payload = {
        "transactions": transactions,
        "include_explanation": include_explanation
    }
    
    try:
        response = requests.post(
            endpoint,
            json=payload,
            timeout=API_TIMEOUT * 2  # Longer timeout for batch processing
        )
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ Batch API Error - Status: {response.status_code}")
            print(f"Response: {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ Batch Request Error: {e}")
        return None

def simulate_batch_fraud_detection(transactions, include_explanation=False):
    """Simulate batch fraud detection"""
    
    results = []
    total_start_time = time.time()
    
    print(f"🔄 Processing {len(transactions)} transactions in batch...")
    
    for transaction in transactions:
        result = simulate_fraud_detection(transaction, include_explanation)
        results.append(result)
    
    total_processing_time = (time.time() - total_start_time) * 1000
    
    print(f"✅ Batch processing completed in {total_processing_time:.1f}ms")
    print(f"📊 Average processing time per transaction: {total_processing_time/len(transactions):.1f}ms")
    
    return results

# Generate batch of transactions for testing
def generate_batch_transactions(n_transactions=25):
    """Generate a batch of test transactions"""
    
    transactions = []
    product_types = ["pix", "credit_card", "ted", "loan"]
    channels = ["mobile_app", "web_browser", "atm"]
    
    for i in range(n_transactions):
        # Generate varied transaction amounts
        if i % 10 == 0:  # Some high-value transactions
            amount = np.random.uniform(10000, 50000)
        elif i % 5 == 0:  # Some medium-value transactions
            amount = np.random.uniform(1000, 5000)
        else:  # Most are low-value
            amount = np.random.uniform(10, 1000)
        
        transaction = {
            "transaction_id": f"batch_txn_{i:03d}",
            "customer_id": f"cust_{np.random.randint(100000, 999999)}",
            "product_type": np.random.choice(product_types),
            "amount": round(amount, 2),
            "currency": "BRL",
            "channel": np.random.choice(channels),
            "timestamp": (datetime.now() - timedelta(minutes=np.random.randint(0, 1440))).isoformat(),
            "device_id": f"device_{np.random.randint(1000, 9999)}",
            "ip_address": f"192.168.{np.random.randint(1, 255)}.{np.random.randint(1, 255)}"
        }
        
        # Add product-specific fields
        if transaction["product_type"] in ["pix", "ted"]:
            transaction["beneficiary_id"] = f"benef_{np.random.randint(10000, 99999)}"
        elif transaction["product_type"] == "credit_card":
            transaction["merchant_id"] = f"merchant_{np.random.randint(1000, 9999)}"
            transaction["merchant_category"] = np.random.choice(["5411", "5541", "5812", "5999"])
        
        transactions.append(transaction)
    
    return transactions

# Generate and process batch
batch_transactions = generate_batch_transactions(25)

print("🔄 Batch Fraud Detection Example")
print("=" * 40)
print(f"📦 Generated {len(batch_transactions)} transactions for batch processing")

# Show sample of transactions
print(f"\n📋 Sample Transactions:")
for i, txn in enumerate(batch_transactions[:3]):
    print(f"  {i+1}. {txn['transaction_id']}: {txn['product_type']} - R$ {txn['amount']:,.2f}")

# Process batch
if api_available:
    batch_results = detect_fraud_batch(batch_transactions)
else:
    batch_results = simulate_batch_fraud_detection(batch_transactions)

if batch_results:
    # Analyze results
    risk_levels = [result['risk_level'] for result in batch_results]
    actions = [result['action'] for result in batch_results]
    processing_times = [result['processing_time_ms'] for result in batch_results]
    
    print(f"\n📊 Batch Processing Results:")
    print(f"   Total Transactions: {len(batch_results)}")
    print(f"   Risk Level Distribution:")
    for level in ['low', 'medium', 'high']:
        count = risk_levels.count(level)
        percentage = (count / len(risk_levels)) * 100
        print(f"     {level.capitalize()}: {count} ({percentage:.1f}%)")
    
    print(f"   Action Distribution:")
    for action in ['approve', 'challenge', 'reject']:
        count = actions.count(action)
        percentage = (count / len(actions)) * 100
        print(f"     {action.capitalize()}: {count} ({percentage:.1f}%)")
    
    print(f"   Performance Metrics:")
    print(f"     Avg Processing Time: {np.mean(processing_times):.1f}ms")
    print(f"     Min Processing Time: {np.min(processing_times):.1f}ms")
    print(f"     Max Processing Time: {np.max(processing_times):.1f}ms")
    print(f"     P95 Processing Time: {np.percentile(processing_times, 95):.1f}ms")
